Best so far but old features
Use stratified kfold
This only use feature that we use in feature importance
Not include ss

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import random

2025-02-21 16:45:59.346470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740156359.368419  810749 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740156359.374758  810749 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 16:45:59.400819: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)  # This sets all random seeds in keras
tf.config.experimental.enable_op_determinism()  # For complete reproducibility

In [3]:
class SimpleGNNLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.node_update = tf.keras.layers.Dense(units)
        self.message = tf.keras.layers.Dense(units)
        
    def call(self, inputs):
        node_features, edge_index, edge_features, node_mask, edge_mask = inputs
        
        # Basic message passing
        edge_index = tf.cast(edge_index, tf.int32)
        source_nodes = tf.gather(node_features, edge_index[..., 0], batch_dims=1)
        
        # Simple messages
        messages = self.message(source_nodes)
        messages = messages * tf.expand_dims(edge_mask, -1)
        
        # Aggregate messages (sum)
        batch_size = tf.shape(node_features)[0]
        num_nodes = tf.shape(node_features)[1]
        
        # Create batch indices for scatter
        batch_range = tf.range(batch_size, dtype=tf.int32)
        batch_range = tf.expand_dims(batch_range, -1)
        batch_range = tf.tile(batch_range, [1, tf.shape(edge_index)[1]])
        
        scatter_indices = tf.stack([
            tf.reshape(batch_range, [-1]),
            tf.reshape(edge_index[..., 1], [-1])
        ], axis=1)
        
        messages = tf.reshape(messages, [-1, self.units])
        output_shape = tf.stack([batch_size, num_nodes, self.units])
        aggregated = tf.zeros(output_shape)
        aggregated = tf.tensor_scatter_nd_add(aggregated, scatter_indices, messages)
        
        # Update node features
        updated = self.node_update(tf.concat([node_features, aggregated], axis=-1))
        updated = updated * tf.expand_dims(node_mask, -1)
        
        return tf.nn.relu(updated)

In [4]:
class GCNLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units
        self.dense = tf.keras.layers.Dense(units, use_bias=False)
        
    def call(self, inputs):
        node_features, edge_index, edge_features, node_mask, edge_mask = inputs
        
        edge_index = tf.cast(edge_index, tf.int32)
        source_nodes = tf.gather(node_features, edge_index[..., 0], batch_dims=1)
        target_nodes = tf.gather(node_features, edge_index[..., 1], batch_dims=1)
        
        # Normalize by degree (approximated by edge mask sum)
        degree = tf.reduce_sum(edge_mask, axis=-1, keepdims=True)
        degree = tf.maximum(degree, 1.0)  # Avoid division by zero
        norm = tf.math.rsqrt(degree)
        
        # GCN propagation
        messages = source_nodes * tf.expand_dims(norm, -1)
        messages = messages * tf.expand_dims(edge_mask, -1)
        
        # Aggregate
        batch_size = tf.shape(node_features)[0]
        num_nodes = tf.shape(node_features)[1]
        
        batch_range = tf.range(batch_size, dtype=tf.int32)
        batch_range = tf.expand_dims(batch_range, -1)
        batch_range = tf.tile(batch_range, [1, tf.shape(edge_index)[1]])
        
        scatter_indices = tf.stack([
            tf.reshape(batch_range, [-1]),
            tf.reshape(edge_index[..., 1], [-1])
        ], axis=1)
        
        messages = tf.reshape(messages, [-1, node_features.shape[-1]])
        output_shape = tf.stack([batch_size, num_nodes, node_features.shape[-1]])
        aggregated = tf.zeros(output_shape)
        aggregated = tf.tensor_scatter_nd_add(aggregated, scatter_indices, messages)
        
        # Linear transformation
        output = self.dense(aggregated)
        output = output * tf.expand_dims(node_mask, -1)
        
        return tf.nn.relu(output)

In [5]:
class GATLayer(tf.keras.layers.Layer):
    def __init__(self, units, num_heads=4):
        super().__init__()
        self.units = units
        self.num_heads = num_heads
        self.head_dim = units // num_heads
        
        self.query = tf.keras.layers.Dense(units)
        self.key = tf.keras.layers.Dense(units)
        self.value = tf.keras.layers.Dense(units)
        self.combine = tf.keras.layers.Dense(units)
        
    def call(self, inputs):
        node_features, edge_index, edge_features, node_mask, edge_mask = inputs
        
        edge_index = tf.cast(edge_index, tf.int32)
        batch_size = tf.shape(node_features)[0]
        num_nodes = tf.shape(node_features)[1]
        
        # Multi-head attention
        q = self.query(node_features)  # [batch, nodes, units]
        k = self.key(node_features)    # [batch, nodes, units]
        v = self.value(node_features)  # [batch, nodes, units]
        
        # Reshape for multiple heads
        q = tf.reshape(q, [batch_size, num_nodes, self.num_heads, self.head_dim])
        k = tf.reshape(k, [batch_size, num_nodes, self.num_heads, self.head_dim])
        v = tf.reshape(v, [batch_size, num_nodes, self.num_heads, self.head_dim])
        
        # Get source and target nodes
        source_nodes = tf.gather(k, edge_index[..., 0], batch_dims=1)  # [batch, edges, heads, head_dim]
        target_nodes = tf.gather(q, edge_index[..., 1], batch_dims=1)  # [batch, edges, heads, head_dim]
        
        # Compute attention scores
        scores = tf.reduce_sum(source_nodes * target_nodes, axis=-1)  # [batch, edges, heads]
        scores = tf.nn.softmax(scores, axis=1)
        scores = scores * tf.expand_dims(edge_mask, -1)
        
        # Apply attention to values
        values = tf.gather(v, edge_index[..., 0], batch_dims=1)
        messages = tf.expand_dims(scores, -1) * values
        
        # Aggregate messages
        batch_range = tf.range(batch_size, dtype=tf.int32)
        batch_range = tf.expand_dims(batch_range, -1)
        batch_range = tf.tile(batch_range, [1, tf.shape(edge_index)[1]])
        
        scatter_indices = tf.stack([
            tf.reshape(batch_range, [-1]),
            tf.reshape(edge_index[..., 1], [-1])
        ], axis=1)
        
        messages = tf.reshape(messages, [-1, self.num_heads * self.head_dim])
        output_shape = tf.stack([batch_size, num_nodes, self.num_heads * self.head_dim])
        aggregated = tf.zeros(output_shape)
        aggregated = tf.tensor_scatter_nd_add(aggregated, scatter_indices, messages)
        
        # Combine heads
        output = self.combine(aggregated)
        output = output * tf.expand_dims(node_mask, -1)
        
        return tf.nn.relu(output)

In [6]:
def create_edge_features(i, j, dist_map, sequence, ss_string, sasa_vals, K_POS=16):
    """
    Create edge features with proper padding handling using sequence
    Returns a fixed-size numpy array
    """
    edge_features = np.zeros(18, dtype=np.float32)  # Pre-allocate fixed size array
    distance = dist_map[i,j]
    
    # Check if either position is padded using sequence
    is_i_padded = sequence[i] == '-'
    is_j_padded = sequence[j] == '-'
    
    if is_i_padded or is_j_padded:
        return edge_features  # Return zero array for padded positions
    
    feature_idx = 0
    
    # 1. Distance Features
    # Distance bins (4 features)
    if distance <= 4.0:
        edge_features[0] = 1.0
    elif distance <= 8.0:
        edge_features[1] = 1.0
    elif distance <= 12.0:
        edge_features[2] = 1.0
    else:
        edge_features[3] = 1.0
    feature_idx += 4
    
    # Continuous distance feature (1 feature)
    edge_features[feature_idx] = 1/distance
    feature_idx += 1
    
    # 2. Sequential Features (2 features)
    seq_dist = abs(i - j)
    edge_features[feature_idx] = float(seq_dist == 1)  # Is sequential
    edge_features[feature_idx + 1] = seq_dist / 32.0  # Normalized distance
    feature_idx += 2
    
    # 3. K-relative Features (2 features)
    edge_features[feature_idx] = float(i == K_POS or j == K_POS)  # Is K-connected
    edge_features[feature_idx + 1] = min(abs(i - K_POS), abs(j - K_POS)) / 16.0  # Min distance to K
    feature_idx += 2
    
    # 4. Secondary Structure Interaction (6 features)
    ss_pairs = ['HH', 'HE', 'HL', 'EE', 'EL', 'LL']
    ss_pair = ''.join(sorted([ss_string[i], ss_string[j]]))
    for idx, pair in enumerate(ss_pairs):
        edge_features[feature_idx + idx] = float(ss_pair == pair)
    feature_idx += 6
    
    # 5. SASA Interaction (2 features)
    edge_features[feature_idx] = abs(sasa_vals[i] - sasa_vals[j])  # SASA difference
    edge_features[feature_idx + 1] = (sasa_vals[i] + sasa_vals[j]) / 2  # SASA average
    
    return edge_features

In [7]:
def calculate_feature_size():
    """Calculate the total size of node features"""
    # Backbone angles (phi, psi, omega) - each has sin and cos
    backbone_size = 3 * 2

    # SASA - single value
    sasa_size = 1

    # SS one-hot encoding (H, E, L)
    ss_size = 3

    # plDDT score - single value
    plddt_size = 1

    # Chi angles (chi1-4) - each has sin and cos
    chi_size = 4 * 2

    # K-relative features
    # - Distance to K (normalized)
    # - Is K position
    # - Is K amino acid
    k_relative_size = 3

    total_size = backbone_size + sasa_size + ss_size + plddt_size + chi_size + k_relative_size
    return total_size

In [8]:
def prepare_gnn_data(df, threshold=8.0, max_nodes=33, max_edges=300):
    """
    Prepare node and edge features using sequence for padding detection
    """
    node_features_list = []
    edge_indices_list = []
    edge_features_list = []
    node_masks_list = []
    edge_masks_list = []
    
    K_POS = 16
    
    print("Processing samples for GNN...")
    for idx, row in df.iterrows():
        sequence = row['sequence']
        # Use sequence for padding detection
        is_padded = [pos for pos, aa in enumerate(sequence) if aa == '-']
        
        # Pre-allocate node features array
        sample_nodes = np.zeros((33, 23), dtype=np.float32)
        
        for pos in range(33):
            if pos in is_padded:
                continue  # Keep zeros for padded positions
            
            feature_idx = 0
            
            # Backbone angles
            for angle in ['phi', 'psi', 'omega']:
                angle_vals = np.array(eval(row[angle]))
                angle_rad = np.pi * angle_vals[pos] / 180.0
                sample_nodes[pos, feature_idx:feature_idx+2] = [np.sin(angle_rad), np.cos(angle_rad)]
                feature_idx += 2
            
            # SASA
            sasa_vals = np.array(eval(row['sasa']))
            sample_nodes[pos, feature_idx] = sasa_vals[pos]
            feature_idx += 1
            
            # SS
            ss_val = row['ss'][pos]
            ss_onehot = [1 if ss_val == ss_type else 0 for ss_type in 'HEL']
            sample_nodes[pos, feature_idx:feature_idx+3] = ss_onehot
            feature_idx += 3
            
            # plDDT
            plddt_vals = np.array(eval(row['plDDT']))
            sample_nodes[pos, feature_idx] = plddt_vals[pos]
            feature_idx += 1
            
            # Chi angles
            for chi in ['chi1', 'chi2', 'chi3', 'chi4']:
                chi_vals = np.array(eval(row[chi]))
                chi_rad = np.pi * chi_vals[pos] / 180.0
                sample_nodes[pos, feature_idx:feature_idx+2] = [np.sin(chi_rad), np.cos(chi_rad)]
                feature_idx += 2
            
            # K-relative features
            sample_nodes[pos, feature_idx] = abs(pos - K_POS) / 16.0  # Distance to K
            feature_idx += 1
            sample_nodes[pos, feature_idx] = float(pos == K_POS)  # Is K position
            feature_idx += 1
            sample_nodes[pos, feature_idx] = float(sequence[pos] == 'K')  # Is K amino acid
        
        # Create edges
        dist_map = np.array(eval(row['distance_map'])).reshape(33, 33)
        edges = []
        edge_attrs = []
        
        # Pre-calculate SASA values
        sasa_vals = np.array(eval(row['sasa']))
        
        for i in range(33):
            if i in is_padded:
                continue
                
            for j in range(33):
                if j in is_padded or i == j:
                    continue
                    
                if dist_map[i,j] != -1 and dist_map[i,j] < threshold:
                    edges.append([i, j])
                    edge_attrs.append(
                        create_edge_features(
                            i, j, dist_map, sequence, row['ss'], 
                            sasa_vals, K_POS
                        )
                    )
        
        # Pad or truncate edges
        if len(edges) == 0:
            # Handle case with no edges
            edges = np.zeros((max_edges, 2), dtype=np.float32)
            edge_attrs = np.zeros((max_edges, 18), dtype=np.float32)
            edge_mask = np.zeros(max_edges, dtype=np.float32)
        else:
            if len(edges) > max_edges:
                # Prioritize K-connected edges
                k_connected = [(idx, edge) for idx, edge in enumerate(edges) 
                             if K_POS in edge]
                other_edges = [(idx, edge) for idx, edge in enumerate(edges) 
                             if K_POS not in edge]
                
                k_connected.sort(key=lambda x: edge_attrs[x[0]][4], reverse=True)
                other_edges.sort(key=lambda x: edge_attrs[x[0]][4], reverse=True)
                
                keep_indices = [idx for idx, _ in (k_connected + other_edges)][:max_edges]
                edges = [edges[i] for i in keep_indices]
                edge_attrs = [edge_attrs[i] for i in keep_indices]
                
                # Convert to fixed-size arrays
                edges = np.array(edges, dtype=np.float32)
                edge_attrs = np.array(edge_attrs, dtype=np.float32)
                edge_mask = np.ones(max_edges, dtype=np.float32)
            else:
                # Pad arrays
                num_edges = len(edges)
                pad_edges = np.zeros((max_edges - num_edges, 2), dtype=np.float32)
                pad_attrs = np.zeros((max_edges - num_edges, 18), dtype=np.float32)
                
                edges = np.concatenate([np.array(edges, dtype=np.float32), pad_edges], axis=0)
                edge_attrs = np.concatenate([np.array(edge_attrs, dtype=np.float32), pad_attrs], axis=0)
                edge_mask = np.concatenate([
                    np.ones(num_edges, dtype=np.float32),
                    np.zeros(max_edges - num_edges, dtype=np.float32)
                ])
        
        # Create node mask
        node_mask = np.array([0.0 if pos in is_padded else 1.0 for pos in range(33)], dtype=np.float32)
        
        node_features_list.append(sample_nodes)
        edge_indices_list.append(edges)
        edge_features_list.append(edge_attrs)
        node_masks_list.append(node_mask)
        edge_masks_list.append(edge_mask)
        
        if idx % 1000 == 0:
            print(f"Processed {idx}/{len(df)} samples")
    
    # Convert to numpy arrays with consistent shapes
    node_features = np.stack(node_features_list, axis=0)
    edge_indices = np.stack(edge_indices_list, axis=0)
    edge_features = np.stack(edge_features_list, axis=0)
    node_masks = np.stack(node_masks_list, axis=0)
    edge_masks = np.stack(edge_masks_list, axis=0)
    
    print(f"\nFinal shapes:")
    print(f"Node features: {node_features.shape}")
    print(f"Edge indices: {edge_indices.shape}")
    print(f"Edge features: {edge_features.shape}")
    print(f"Node masks: {node_masks.shape}")
    print(f"Edge masks: {edge_masks.shape}")
    
    return node_features, edge_indices, edge_features, node_masks, edge_masks

In [9]:
def create_hybrid_model(gnn_type='simple', seq_length=33, node_features=23, max_nodes=33, max_edges=300):
    """
    Create hybrid model with sequence CNN and choice of GNN
    gnn_type: 'simple', 'gcn', or 'gat'
    """
    regularizer = tf.keras.regularizers.l2(0.01)
    
    # Sequence track (CNN)
    seq_input = tf.keras.layers.Input(shape=(seq_length,), name='sequence_input')
    x_seq = tf.keras.layers.Embedding(21, 21)(seq_input)
    x_seq = tf.keras.layers.Reshape((seq_length, 21, 1))(x_seq)
    x_seq = tf.keras.layers.Conv2D(32, kernel_size=(17, 3), activation='relu', padding='valid')(x_seq)
    x_seq = tf.keras.layers.BatchNormalization()(x_seq)
    x_seq = tf.keras.layers.Dropout(0.4)(x_seq)
    x_seq = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x_seq)
    x_seq = tf.keras.layers.Flatten()(x_seq)
    x_seq = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizer)(x_seq)
    
    # GNN track
    node_input = tf.keras.layers.Input(shape=(max_nodes, node_features), dtype=tf.float32, name='node_features')
    edge_index_input = tf.keras.layers.Input(shape=(max_edges, 2), dtype=tf.float32, name='edge_index')
    edge_feature_input = tf.keras.layers.Input(shape=(max_edges, 18), dtype=tf.float32, name='edge_features')
    node_mask_input = tf.keras.layers.Input(shape=(max_nodes,), dtype=tf.float32, name='node_mask')
    edge_mask_input = tf.keras.layers.Input(shape=(max_edges,), dtype=tf.float32, name='edge_mask')
    
    # GNN layers based on type
    x_gnn = node_input
    if gnn_type == 'simple':
        GNNLayer = SimpleGNNLayer
    elif gnn_type == 'gcn':
        GNNLayer = GCNLayer
    else:  # gat
        GNNLayer = GATLayer
    
    # Single GNN layer
    x_gnn = GNNLayer(32)([x_gnn, edge_index_input, edge_feature_input, node_mask_input, edge_mask_input])
    x_gnn = tf.keras.layers.GlobalAveragePooling1D()(x_gnn)
    
    # Combine tracks
    combined = tf.keras.layers.Concatenate()([x_seq, x_gnn])
    
    # Final classification
    x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizer)(combined)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.Model(
        inputs=[seq_input, node_input, edge_index_input, edge_feature_input, node_mask_input, edge_mask_input],
        outputs=outputs
    )
    
    return model

In [10]:
class AttentionPooling(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.attention_dense = tf.keras.layers.Dense(1, activation='sigmoid')
    
    def call(self, inputs, mask=None):
        x, node_mask = inputs
        attention_weights = self.attention_dense(x)
        attention_weights = attention_weights * tf.expand_dims(node_mask, -1)
        return tf.reduce_sum(x * attention_weights, axis=1)

In [11]:
def prepare_sequence_data(df):
    """Convert sequences to integer encoding"""
    alphabet = 'ARNDCQEGHILKMFPSTWYV-'
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    
    sequences = df['sequence'].values
    encodings = []
    
    for seq in sequences:
        try:
            integer_encoded = [char_to_int[char] for char in seq]
            encodings.append(integer_encoded)
        except Exception as e:
            print(f"Error processing sequence: {e}")
            continue
    
    return np.array(encodings, dtype=np.int32)

In [13]:
def print_metrics(y_true, y_pred):
    """Print comprehensive evaluation metrics"""
    cm = confusion_matrix(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    sensitivity = cm[1][1]/(cm[1][1]+cm[1][0])
    specificity = cm[0][0]/(cm[0][0]+cm[0][1])
    
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("Confusion Matrix:")
    print(cm)

In [14]:
def train_with_cv(train_df, test_df, threshold=8.0, gnn_type=None):
    """
    Training function with cross-validation
    Args:
        train_df: training dataframe
        test_df: test dataframe
        threshold: distance threshold for edge creation
        gnn_type: 'simple', 'gcn', 'gat', or None (will try all)
    """
    # Prepare data once
    print("Preparing sequence data...")
    
    train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

    X_train_seq = prepare_sequence_data(train_df)
    X_test_seq = prepare_sequence_data(test_df)
    
    print("Preparing GNN data...")
    (train_node_features, train_edge_indices, train_edge_features,
     train_node_masks, train_edge_masks) = prepare_gnn_data(train_df, threshold=threshold)
    
    (test_node_features, test_edge_indices, test_edge_features,
     test_node_masks, test_edge_masks) = prepare_gnn_data(test_df, threshold=threshold)
    
    y_train = train_df['label'].values
    y_test = test_df['label'].values
    
    # Print class distribution
    print("\nClass distribution:")
    print("Train:", np.bincount(y_train))
    print("Test:", np.bincount(y_test))
    
    # Define which GNN types to try
    if gnn_type is None:
        gnn_types = ['simple', 'gcn', 'gat']
    else:
        gnn_types = [gnn_type]
    
    # Store results for all types
    all_results = {}
    
    for gnn_type in gnn_types:
        print(f"\nTraining {gnn_type.upper()} model")
        
        # Cross-validation
        kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        metrics = {
            'accuracy': [], 'balanced_accuracy': [], 
            'mcc': [], 'sensitivity': [], 'specificity': []
        }
        test_predictions = []
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_seq, y_train), 1):
            print(f"\nFold {fold}/5")
            
            # Calculate class weights
            total = len(train_idx)
            pos = np.sum(y_train[train_idx] == 1)
            neg = np.sum(y_train[train_idx] == 0)
            class_weights = {
                0: total / (2 * neg),
                1: total / (2 * pos)
            }
            
            # Create and compile model
            model = create_hybrid_model(
                gnn_type=gnn_type,
                seq_length=33,
                node_features=train_node_features.shape[-1],
                max_nodes=train_node_features.shape[1],
                max_edges=train_edge_indices.shape[1]
            )
            
            model.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            # Callbacks
            callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=7,
                    restore_best_weights=True
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.5,
                    patience=4,
                    min_lr=1e-6
                )
            ]
            
            # Train
            history = model.fit(
                [
                    X_train_seq[train_idx],
                    train_node_features[train_idx],
                    train_edge_indices[train_idx],
                    train_edge_features[train_idx],
                    train_node_masks[train_idx],
                    train_edge_masks[train_idx]
                ],
                y_train[train_idx],
                validation_data=(
                    [
                        X_train_seq[val_idx],
                        train_node_features[val_idx],
                        train_edge_indices[val_idx],
                        train_edge_features[val_idx],
                        train_node_masks[val_idx],
                        train_edge_masks[val_idx]
                    ],
                    y_train[val_idx]
                ),
                batch_size=32,
                epochs=50,
                callbacks=callbacks,
                class_weight=class_weights,
                verbose=1
            )
            
            # Evaluate on validation set
            val_pred = model.predict(
                [
                    X_train_seq[val_idx],
                    train_node_features[val_idx],
                    train_edge_indices[val_idx],
                    train_edge_features[val_idx],
                    train_node_masks[val_idx],
                    train_edge_masks[val_idx]
                ]
            )
            val_pred_binary = (val_pred > 0.5).astype(int)
            
            # Calculate metrics
            acc = accuracy_score(y_train[val_idx], val_pred_binary)
            balanced_acc = balanced_accuracy_score(y_train[val_idx], val_pred_binary)
            mcc = matthews_corrcoef(y_train[val_idx], val_pred_binary)
            cm = confusion_matrix(y_train[val_idx], val_pred_binary)
            sensitivity = cm[1][1]/(cm[1][1]+cm[1][0]) if (cm[1][1]+cm[1][0]) > 0 else 0
            specificity = cm[0][0]/(cm[0][0]+cm[0][1]) if (cm[0][0]+cm[0][1]) > 0 else 0
            
            metrics['accuracy'].append(acc)
            metrics['balanced_accuracy'].append(balanced_acc)
            metrics['mcc'].append(mcc)
            metrics['sensitivity'].append(sensitivity)
            metrics['specificity'].append(specificity)
            
            # Predict on test set
            test_pred = model.predict(
                [
                    X_test_seq,
                    test_node_features,
                    test_edge_indices,
                    test_edge_features,
                    test_node_masks,
                    test_edge_masks
                ]
            )
            test_predictions.append(test_pred)
            
            # Print fold results
            print(f"\nFold {fold} Results:")
            print(f"Accuracy: {acc:.4f}")
            print(f"Balanced Accuracy: {balanced_acc:.4f}")
            print(f"MCC: {mcc:.4f}")
            print(f"Sensitivity: {sensitivity:.4f}")
            print(f"Specificity: {specificity:.4f}")
            print("Confusion Matrix:")
            print(cm)
        
        # Print average CV results
        print(f"\nAverage Cross-validation Results for {gnn_type.upper()}:")
        cv_results = {}
        for metric in metrics:
            mean = np.mean(metrics[metric])
            std = np.std(metrics[metric])
            print(f"{metric}: {mean:.4f} ± {std:.4f}")
            cv_results[metric] = {'mean': mean, 'std': std}
        
        # Ensemble predictions on test set
        test_pred_avg = np.mean(test_predictions, axis=0)
        test_pred_binary = (test_pred_avg > 0.5).astype(int)
        
        # Print final test results
        print(f"\nFinal Test Set Results for {gnn_type.upper()}:")
        test_results = print_metrics(y_test, test_pred_binary)
        
        # Store results
        all_results[gnn_type] = {
            'cv_results': cv_results,
            'test_results': test_results,
            'model': model
        }
    
    return all_results

In [15]:
if __name__ == "__main__":
    # Example usage
    # train_df = pd.read_csv("../data/processed_features_fixed_train_contactmap_copy.csv")
    # test_df = pd.read_csv("../data/processed_features_fixed_test_contactmap_copy.csv")
    train_df = pd.read_csv("../data/processed_features_fixed_train_contactmap.csv")
    test_df = pd.read_csv("../data/processed_features_fixed_test_contactmap.csv")    
    # Train with different thresholds
    # For a single threshold
    # To try a specific GNN type:
    # results = train_with_cv(train_df, test_df, threshold=8.0, gnn_type='gcn')

    # To try all GNN types:
    results = train_with_cv(train_df, test_df, threshold=8.0)

    # Or try multiple thresholds
    # thresholds = [6.0, 8.0, 10.0, 12.0]
    # for threshold in thresholds:
    #     print(f"\nEvaluating threshold {threshold}Å")
    #     model, history, cv_metrics = train_with_cv(train_df, test_df, threshold=threshold)

Preparing sequence data...
Preparing GNN data...
Processing samples for GNN...
Processed 0/8853 samples
Processed 1000/8853 samples
Processed 2000/8853 samples
Processed 3000/8853 samples
Processed 4000/8853 samples
Processed 5000/8853 samples
Processed 6000/8853 samples
Processed 7000/8853 samples
Processed 8000/8853 samples

Final shapes:
Node features: (8853, 33, 23)
Edge indices: (8853, 300, 2)
Edge features: (8853, 300, 18)
Node masks: (8853, 33)
Edge masks: (8853, 300)
Processing samples for GNN...
Processed 0/2737 samples
Processed 1000/2737 samples
Processed 2000/2737 samples

Final shapes:
Node features: (2737, 33, 23)
Edge indices: (2737, 300, 2)
Edge features: (2737, 300, 18)
Node masks: (2737, 33)
Edge masks: (2737, 300)

Class distribution:
Train: [4261 4592]
Test: [2497  240]

Training SIMPLE model

Fold 1/5


I0000 00:00:1740156621.594509  810749 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:00:06.0, compute capability: 7.0


Epoch 1/50


2025-02-21 16:50:24.211377: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_21}}
E0000 00:00:1740156626.249307  810749 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:174015

219/222 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5339 - loss: 1.3410

...

56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

Fold 4 Results:
Accuracy: 0.7311
Balanced Accuracy: 0.7276
MCC: 0.4645
Sensitivity: 0.8203
Specificity: 0.6350
Confusion Matrix:
[[541 311]
 [165 753]]

Fold 5/5
Epoch 1/50
221/222 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5483 - loss: 1.8118

2025-02-21 17:04:37.316419: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_20}}


222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.5484 - loss: 1.8095 - val_accuracy: 0.4814 - val_loss: 1.1509 - learning_rate: 0.0010
Epoch 2/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6221 - loss: 1.0553 - val_accuracy: 0.4819 - val_loss: 1.0123 - learning_rate: 0.0010
Epoch 3/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.6775 - loss: 0.8313 - val_accuracy: 0.4814 - val_loss: 1.0824 - learning_rate: 0.0010
Epoch 4/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7084 - loss: 0.7126 - val_accuracy: 0.4921 - val_loss: 1.0664 - learning_rate: 0.0010
Epoch 5/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7243 - loss: 0.6489 - val_accuracy: 0.5136 - val_loss: 0.9487 - learning_rate: 0.0010
Epoch 6/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7398 - loss: 0.6059 - val_accuracy: 0.6492 - val_loss: 0.6871 - learning_rate: 0.0010
Epoch 7/50
222/222 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7494 - loss: 0.5813 - val_

2025-02-21 17:05:25.966958: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_19}}


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
86/86 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

Fold 5 Results:
Accuracy: 0.7181
Balanced Accuracy: 0.7206
MCC: 0.4441
Sensitivity: 0.6525
Specificity: 0.7887
Confusion Matrix:
[[672 180]
 [319 599]]

Average Cross-validation Results for GAT:
accuracy: 0.7266 ± 0.0084
balanced_accuracy: 0.7261 ± 0.0071
mcc: 0.4551 ± 0.0153
sensitivity: 0.7413 ± 0.0580
specificity: 0.7109 ± 0.0501

Final Test Set Results for GAT:
Accuracy: 0.7318
Balanced Accuracy: 0.7382
MCC: 0.2910
Sensitivity: 0.7458
Specificity: 0.7305
Confusion Matrix:
[[1824  673]
 [  61  179]]
